# Replace exchange-correlation by Dirac exchange.

In [1]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [2]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
import matplotlib.pyplot as plt

In [3]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'"
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['4 4 4 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [4]:
driver=Driver(qe_options=qe_options, logfile='tmp.out')

In [5]:
driver.scf()

-137.91449174774482

In [6]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

In [7]:
def diracx(rho, volume):
    v = -np.cbrt(3.0/np.pi*rho)
    e = np.sum((v*rho))*(3.0/4.0)*volume/rho.size
    return v*2,e*2

In [8]:
for i in range(60):
    volume = driver.get_volume()
    extpot, ex = diracx(driver.get_density(), volume)
    driver.set_external_potential(potential=extpot, extene=ex, exttype=4)
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

Iter:  0  - Conv:  0.0841029689989849
Iter:  1  - Conv:  0.0012522874439220622
Iter:  2  - Conv:  2.8973165131754847e-05
Iter:  3  - Conv:  6.445561502984867e-08


In [9]:
driver.get_energy()

-136.14828243776793